In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
import h5py
from collections import Counter
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn import tree
from sklearn.svm import LinearSVC
from sklearn.utils import shuffle
import nltk
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

stemmer = SnowballStemmer("english", ignore_stopwords=True)
lemmatizer = WordNetLemmatizer()
stopwords = stopwords.words('english')
np.set_printoptions(threshold=np.nan)

In [2]:
# Load data
IMDB_train = pd.read_csv('./Dataset/Input/IMDB-train.txt', sep='\t', encoding='latin-1', header=None)
IMDB_train_y = IMDB_train[:][1]
IMDB_valid = pd.read_csv('./Dataset/Input/IMDB-valid.txt', sep='\t', encoding='latin-1', header=None)
IMDB_valid_y = IMDB_valid[:][1]
IMDB_test = pd.read_csv('./Dataset/Input/IMDB-test.txt', sep='\t', encoding='latin-1', header=None)
IMDB_test_y = IMDB_test[:][1]
stemmer = SnowballStemmer("english", ignore_stopwords=True)

print("Data loaded.")

Data loaded.


In [3]:
frames = [IMDB_train, IMDB_valid]
frames_y = [IMDB_train_y, IMDB_valid_y]
IMDB_train = pd.concat(frames)
IMDB_train_y = pd.concat(frames_y)

In [4]:
# IMDB_train = IMDB_train[:][0]
# IMDB_test = IMDB_test[:][0]

In [5]:
IMDB_train.shape

(25000, 2)

In [6]:
IMDB_test_y.shape

(25000,)

# Preprocessing

In [7]:
def preprocessing(data):
    new_data = []
    #i = 0
    for sentence in (data[:][0]):
        #clean = re.compile('<.*?>')
        new_sentence = re.sub('<.*?>', '', sentence) # remove HTML tags
        new_sentence = re.sub(r'[^\w\s]', '', new_sentence) # remove punctuation
        new_sentence = new_sentence.lower() # convert to lower case
        if new_sentence != '':
            new_data.append(new_sentence)
    return new_data

In [8]:
"""
def rm_stopwords(data):
    new_data = []
    for sent in data:
        new_data.append([w for w in sent if w not in stopwords])
    return new_data
"""

'\ndef rm_stopwords(data):\n    new_data = []\n    for sent in data:\n        new_data.append([w for w in sent if w not in stopwords])\n    return new_data\n'

In [9]:
"""
<<<<IGNORE THESE FOR NOW>>>>

def tokenize(data):
    new_data = []
    for sentence in (data):
        new_sentence = nltk.word_tokenize(sentence)
        new_data.append(new_sentence)
    return new_data        

def stem_lem(data):
    new_data = []
    for sent in data:
        this_sent = []
        for w in test3:
            w = stemmer.stem(w)
            w = lemmatizer.lemmatize(w)
            this_sent.append(stemmer.stem(w))
        new_data.append(this_sent)
    return new_data
"""

'\n<<<<IGNORE THESE FOR NOW>>>>\n\ndef tokenize(data):\n    new_data = []\n    for sentence in (data):\n        new_sentence = nltk.word_tokenize(sentence)\n        new_data.append(new_sentence)\n    return new_data        \n\ndef stem_lem(data):\n    new_data = []\n    for sent in data:\n        this_sent = []\n        for w in test3:\n            w = stemmer.stem(w)\n            w = lemmatizer.lemmatize(w)\n            this_sent.append(stemmer.stem(w))\n        new_data.append(this_sent)\n    return new_data\n'

In [10]:
IMDB_train = preprocessing(IMDB_train)
#IMDB_valid = preprocessing(IMDB_valid)
IMDB_test = preprocessing(IMDB_test)

In [11]:
#IMDB_train_tok = tokenize(IMDB_train)
#IMDB_valid_tok = tokenize(IMDB_valid)
#IMDB_test_tok = tokenize(IMDB_test)

In [12]:
#IMDB_train_sl = stem_lem(IMDB_train_tok)
#IMDB_valid_sl = stem_lem(IMDB_valid_tok)
#IMDB_test_sl = stem_lem(IMDB_test_tok)

In [13]:
#IMDB_train_stop = rm_stopwords(IMDB_train_sl)
#IMDB_valid_stop = rm_stopwords(IMDB_valid_sl)
#IMDB_test_stop = rm_stopwords(IMDB_test_sl)

# Bag of n-gram 

In [14]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [15]:
unigram = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(1, 1), stop_words='english', max_features =30000)
bigram = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(2, 2), stop_words='english', max_features =30000)
trigram = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(3, 3), stop_words='english', max_features =30000)

In [16]:
train_unigram = unigram.fit_transform(IMDB_train).toarray()
test_unigram = unigram.transform(IMDB_test).toarray()

/Users/AbhijayGupta/anaconda3/envs/myenv/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  sorted(inconsistent))


In [17]:
train_bigram = bigram.fit_transform(IMDB_train).toarray()
test_bigram = bigram.transform(IMDB_test).toarray()

/Users/AbhijayGupta/anaconda3/envs/myenv/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  sorted(inconsistent))


In [18]:
train_trigram = trigram.fit_transform(IMDB_train).toarray()
test_trigram = trigram.transform(IMDB_test).toarray()

/Users/AbhijayGupta/anaconda3/envs/myenv/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  sorted(inconsistent))


In [27]:
type(test_bigram)
test_trigram.shape
# np.save('./train_unigram', train_unigram)

(25000, 30000)

In [ ]:
# with h5py.File('train_unigram.hdf5', 'w') as f:
#     dset = f.create_dataset("default", data=train_unigram)

In [ ]:
#pd.DataFrame(train_unigram).to_csv('./train_unigram.csv', index = False, header = False)
#pd.DataFrame.from_records(train_bigram).to_csv('./train_bigram.csv', index = False, header = False)
#pd.DataFrame.from_records(train_trigram).to_csv('./train_trigram.csv', index = False, header = False)


In [ ]:
unigram_w_sw = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(1, 1), stop_words=None, max_features =30000)
bigram_w_sw = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(2, 2), stop_words=None, max_features =30000)
trigram_w_sw = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(3, 3), stop_words=None, max_features =30000)

In [ ]:
train_unigram_w_sw = unigram_w_sw.fit_transform(IMDB_train).toarray()
test_unigram_w_sw = unigram_w_sw.transform(IMDB_test).toarray()

In [ ]:
train_bigram_w_sw = bigram_w_sw.fit_transform(IMDB_train).toarray()
test_bigram_w_sw = bigram_w_sw.transform(IMDB_test).toarray()

In [ ]:
train_trigram_w_sw = trigram_w_sw.fit_transform(IMDB_train).toarray()
test_trigram_w_sw = trigram_w_sw.transform(IMDB_test).toarray()

In [ ]:
train_trigram.shape

In [ ]:
# pd.DataFrame.from_records(train_unigram_w_sw).to_csv('./train_unigram_w_sw.csv', index = False, header = False)
# pd.DataFrame.from_records(train_bigram_w_sw).to_csv('./train_bigram_w_sw.csv', index = False, header = False)
# pd.DataFrame.from_records(train_trigram_w_sw).to_csv('./train_trigram_w_sw.csv', index = False, header = False)

# Linear SVM

## Without stop words

## Unigram 

In [ ]:
# imdb_linearsvm_clf = imdb_linearsvm_clf.fit(train_unigram, IMDB_train_y)

In [ ]:
# imdb_linearsvm_best_params = imdb_linearsvm_clf.best_params_
# imdb_linearsvm_best_params

In [ ]:
# imdb_linearsvm_clf = LinearSVC(tol=imdb_linearsvm_best_params['tol'],
#                                     C=imdb_linearsvm_best_params['C'])

In [ ]:
# imdb_linearsvm_clf = imdb_linearsvm_clf.fit(train_unigram, IMDB_train_y)

# imdb_linearsvm_train_pred = imdb_linearsvm_clf.predict(train_unigram)
# # imdb_linearsvm_valid_pred = imdb_linearsvm_clf.predict(valid_trigram)
# imdb_linearsvm_test_pred = imdb_linearsvm_clf.predict(test_unigram)

# imdb_linearsvm_train_f1 = f1_score(IMDB_train_y, imdb_linearsvm_train_pred, average='micro')
# # imdb_linearsvm_valid_f1 = f1_score(IMDB_valid_y, imdb_linearsvm_valid_pred, average='micro')
# imdb_linearsvm_test_f1 = f1_score(IMDB_test_y, imdb_linearsvm_test_pred, average='micro')

In [ ]:
# print("IMDB Linear SVM Train F1 Score:", imdb_linearsvm_train_f1)
# # print("IMDB Linear SVM Valid F1 Score:", imdb_linearsvm_valid_f1)
# print("IMDB Linear SVM Test F1 Score:", imdb_linearsvm_test_f1)
# print("IMDB Best Linear SVM Parameters: ", imdb_linearsvm_best_params)

In [28]:
def run_linearsvm(imdb_linearsvm_clf, train_input, train_output, test_input, test_output):
    imdb_linearsvm_clf = imdb_linearsvm_clf.fit(train_input, train_output)
    imdb_linearsvm_best_params = imdb_linearsvm_clf.best_params_
    print(imdb_linearsvm_best_params)
    imdb_linearsvm_clf = LinearSVC(tol=imdb_linearsvm_best_params['tol'],
                                    C=imdb_linearsvm_best_params['C'])
    
    imdb_linearsvm_clf = imdb_linearsvm_clf.fit(train_input, train_output)
    
    # make predictions
    imdb_linearsvm_train_pred = imdb_linearsvm_clf.predict(train_input)
    # imdb_linearsvm_valid_pred = imdb_linearsvm_clf.predict(valid_trigram)
    imdb_linearsvm_test_pred = imdb_linearsvm_clf.predict(test_input)

    # calculate f1 score
    imdb_linearsvm_train_f1 = f1_score(train_output, imdb_linearsvm_train_pred, average='micro')
    # imdb_linearsvm_valid_f1 = f1_score(IMDB_valid_y, imdb_linearsvm_valid_pred, average='micro')
    imdb_linearsvm_test_f1 = f1_score(test_output, imdb_linearsvm_test_pred, average='micro')
    
    return [imdb_linearsvm_train_f1, imdb_linearsvm_test_f1, imdb_linearsvm_best_params]

In [45]:
imdb_linearsvm_clf = LinearSVC(max_iter=20000)
list_C = [2*i for i in range(1, 5)]
list_tol = [10**(-i*2) for i in range(1, 4)]

# parameter grid to check against
# for hyperparameter tuning
tuned_parameters = [{'C': list_C, 'tol': list_tol}]
imdb_linearsvm_clf = GridSearchCV(imdb_linearsvm_clf, tuned_parameters, scoring='f1_micro', cv=3, verbose=2, 
                                        n_jobs=-1)

In [46]:
type(train_unigram)

numpy.ndarray

In [47]:
unigram_results = run_linearsvm(imdb_linearsvm_clf, train_unigram, IMDB_train_y, test_unigram, IMDB_test_y)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
print("IMDB Linear SVM Train Unigram F1 Score:", unigram_results[0])
# print("IMDB Linear SVM Valid F1 Score:", imdb_linearsvm_valid_f1)
print("IMDB Linear SVM Test Unigram F1 Score:", unigram_results[1])
print("IMDB Best Linear SVM (Unigram) Parameters: ", unigram_results[2])